<a href="https://colab.research.google.com/github/emilyliublair/Machine-Learning-Projects/blob/main/predicting_health_costs_with_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
#split dataset
train_dataset = dataset[0:int(.8*len(dataset))]
test_dataset = dataset[int(.8*len(dataset)):]

train_labels = train_dataset.pop('expenses')
test_labels = test_dataset.pop('expenses')

print(train_dataset.head())

In [ ]:
train_dataset.age.hist(bins=20)

In [ ]:
train_dataset.sex.value_counts().plot(kind='barh')

In [ ]:
pd.concat([train_dataset, train_labels], axis=1).groupby('sex').expenses.mean().plot(kind='barh')

In [ ]:
#input function
def input_fn(features, labels, training=True, batch_size=256):
  dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
  if training:
    dataset=dataset.shuffle(1000).repeat()
  print(dataset.batch(batch_size))
  return (dataset.batch(batch_size))

In [ ]:
#preprocess data
categorical = ['sex', 'smoker', 'region']
numeric = ['age', 'bmi', 'children']

feature_columns = []
for feature_name in categorical:
  vocab = train_dataset[feature_name].unique()
  feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocab))

for feature_name in numeric:
  feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))

print(feature_columns)

train_dataset = train_dataset.replace({'no':0, 'yes':1,'male':0,'female':1,'southwest':0,'southeast':1,'northwest':2,'northeast':3})
test_dataset = test_dataset.replace({'no':0, 'yes':1,'male':0,'female':1,'southwest':0,'southeast':1,'northwest':2,'northeast':3})
print(train_dataset)

In [ ]:
normalizer = tf.keras.layers.experimental.preprocessing.Normalization()
normalizer.adapt(np.array(train_dataset))

In [ ]:
#create model
model = tf.keras.Sequential([
    normalizer,
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation="relu"),
    layers.Dense(1, activation="relu")
])

model.summary()

In [ ]:
#train model
model.compile(optimizer=tf.keras.optimizers.Adam(.001), loss='mean_absolute_error', metrics=['mae','mse'])

history=model.fit(
    train_dataset, train_labels, epochs=500
)

In [ ]:
#prediciton
model.predict(train_dataset[:10])

In [ ]:
#evaluate model
test_results = model.evaluate(test_dataset, test_labels)
print(test_results)

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
